In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

# Define the path to your training set folder
train_set_folder = r"Train/"  # Replace with the path to your training set folder

# Create empty lists to store image and label data
image_data = []
label_data = []

# Define a dictionary to map folder names to class labels
class_mapping = {
    "Gesture01": 0,
    "Gesture02": 1,
    "Gesture03": 2,
    "Gesture04": 3,
    "Gesture05": 4,
    "Gesture06": 5,
    "Gesture07": 6,
    "Gesture08": 7,
    "Gesture09": 8,
    "Gesture10": 9,
    "Gesture11": 10,
    "Gesture12": 11, 
    "Gesture13": 12, 
    "Gesture14": 13, 
    "Gesture15": 14, 
    "Gesture16": 15,
    # Add mappings for the rest of your folder names and class labels
}

# Iterate over the folders in the training set folder
for class_folder in os.listdir(train_set_folder):
    class_folder_path = os.path.join(train_set_folder, class_folder)
    if os.path.isdir(class_folder_path):
        # Get the class label from the folder name using the mapping
        class_label = class_mapping.get(class_folder)
        if class_label is not None:
            # Load images from the class folder
            for image_file in os.listdir(class_folder_path):
                image_path = os.path.join(class_folder_path, image_file)
                image = Image.open(image_path).convert("L")  # Open the image and convert to grayscale
                image = image.resize((100, 100))  # Resize the image to (28, 28)
                image_data.append(np.array(image))
                label_data.append(class_label)

# Convert the image and label data to NumPy arrays
X_train = np.array(image_data)
y_train = np.array(label_data)

# Normalize pixel values between 0 and 1
X_train = X_train / 255.0

# Convert the labels to one-hot encoded format
num_classes = len(class_mapping)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
#y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Define the path to your test set folder
test_set_folder = r"Test/"  # Replace with the path to your test set folder

# Create empty lists to store test image and label data
test_image_data = []
test_label_data = []

# Iterate over the folders in the test set folder
for class_folder in os.listdir(test_set_folder):
    class_folder_path = os.path.join(test_set_folder, class_folder)
    if os.path.isdir(class_folder_path):
        # Get the class label from the folder name using the mapping
        class_label = class_mapping.get(class_folder)
        if class_label is not None:
            # Load images from the class folder
            for image_file in os.listdir(class_folder_path):
                image_path = os.path.join(class_folder_path, image_file)
                image = Image.open(image_path).convert("L")  # Open the image and convert to grayscale
                image = image.resize((100, 100))  # Resize the image to (28, 28)
                test_image_data.append(np.array(image))
                test_label_data.append(class_label)

# Convert the test image and label data to NumPy arrays
X_test = np.array(test_image_data)
y_test = np.array(test_label_data)

# Normalize pixel values between 0 and 1
X_test = X_test / 255.0

# Convert the test labels to one-hot encoded format
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Define the CNN model
model = keras.Sequential([
    keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='tanh'),
    #keras.layers.add(Dropout(0.5)),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dropout(0.5),
    #keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')  # Assuming you have 18 classes
])

model.summary()

# Compile the model
#optimizer = Adam(learning_rate=0.001)
optimizer = Adam()
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Define the learning rate scheduler callback
#lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.01, patience=3)

# Train the model with the learning rate scheduler callback
history = model.fit(X_train, y_train, batch_size=25, epochs=20, validation_data=(X_test, y_test)) #callbacks=[lr_scheduler])


# Train the model
# history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

print("Train accuracy:", history.history['accuracy'][-1])
print("Val accuracy:", history.history['val_accuracy'][-1])

print("Train loss:", history.history['loss'][-1])
print("Val loss:", history.history['val_loss'][-1])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

# Classify your own dataset
# X_new = np.array(...)  # Replace ... with your own new image data
# X_new = X_new / 255.0
# predictions = model.predict(X_new)
# predicted_labels = np.argmax(predictions, axis=1)

# Print predicted labels for your own dataset
# print('Predicted labels:', predicted_labels)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 98, 98, 64)        640       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 49, 49, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 23, 23, 128)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 21, 21, 128)       147584    
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 10, 10, 128)     